In [149]:
import pandas as pd
import os 
file = "/data/steam_games.csv"
file = os.getcwd() + file
raw_steam_data = pd.read_csv(file, nrows=8000)

In [150]:
raw_steam_data['name'].size #40833 games
names = raw_steam_data['name'] 
names.size #40833 names (possibly not unique) 
types = raw_steam_data['types']
#types[types.duplicated(keep=False)].size

#example of duplicate names -> unsuitable for unique id
for i in range(0, raw_steam_data['name'].size):
    if "The Deer" == raw_steam_data['name'][i]:
        print(raw_steam_data.iloc[i])
        print(i)

In [151]:
#purging unwanted data
purged_steam_data = raw_steam_data.drop(columns=['types', 'desc_snippet', 'achievements','game_description', 'minimum_requirements', 'recommended_requirements'])
purged_steam_data.iloc[5]

url               https://store.steampowered.com/bundle/5699/Gra...
name                     Grand Theft Auto V: Premium Online Edition
recent_reviews                                                  NaN
all_reviews                                                     NaN
release_date                                                    NaN
developer                                            Rockstar North
publisher                                            Rockstar Games
popular_tags                                                    NaN
game_details      Single-player,Multi-player,Downloadable Conten...
languages         English, French, Italian, German, Spanish - Sp...
genre                                              Action,Adventure
mature_content                                                  NaN
original_price                                                  NaN
discount_price                                               $35.18
Name: 5, dtype: object

In [152]:
import re
import math
amount_p = re.compile(r'\d+(,\d+)*')
value_p = re.compile(r'\d+')

def process_review(review):
    #print(review)
    if isinstance(review, str) :
        str_num = amount_p.search(review).group()
        nums = value_p.findall(str_num)
        amt = 0
        if 1 < len(nums):
            for i in range(0, len(nums) - 1):
                prev = nums[i + 1]
                power = len(prev)
                prev = int(prev)
                amt += int(nums[i])*(10**power) + prev
                
        else:
            amt = nums[0]
        
        all_nums = value_p.findall(review)
        if 2 < len(all_nums):
            val = all_nums[len(nums)]
        else:
            val = -1
            
        return [amt, val]
    else: 
        return []
    
def get_element(nums, index):
    if index < len(nums):
        return int(nums[index])
    else:
        return 0

recent_review_numbers = purged_steam_data.apply(lambda x: process_review(x['recent_reviews']), axis=1)
recent_review_numbers


purged_steam_data['amt_rec_revs'] = recent_review_numbers.apply(lambda x: get_element(x, 0))
purged_steam_data['rec_rev_score'] = recent_review_numbers.apply(lambda x: get_element(x, 1))

alltime_review_numbers = purged_steam_data.apply(lambda x: process_review(x['all_reviews']), axis=1)

purged_steam_data['amt_all_revs'] = alltime_review_numbers.apply(lambda x: get_element(x, 0))
purged_steam_data['all_rev_score'] = alltime_review_numbers.apply(lambda x: get_element(x, 1))

purged_steam_data = purged_steam_data.drop(columns=['recent_reviews','all_reviews'])

#keys for new columns:
#'amt_rec_revs' -> the amount of recent reviews
#'rec_rev_score' -> the percentage of the recent reviews
#'amt_all_revs' -> the amount of all reviews
#'all_rev_score' - > the perceent of all reviews


In [153]:
import functools 

def handle_prices(price):
    if isinstance(price, str):
        prices = value_p.findall(price)
        if len(prices):
            price = float(prices[0]) 
            if 1 < len(prices):
                price += float(prices[1])/100
            return price
        else:
            return 0
    else:
        return -1 #return nan

purged_steam_data['original_price'] = purged_steam_data.apply(lambda x: handle_prices(x['original_price']), axis=1)
purged_steam_data['discount_price'] = purged_steam_data.apply(lambda x: handle_prices(x['discount_price']), axis=1)


#value_p.findall(purged_steam_data['original_price'][4])

In [154]:
purged_steam_data

,url,name,release_date,developer,publisher,popular_tags,game_details,languages,genre,mature_content,original_price,discount_price,amt_rec_revs,rec_rev_score,amt_all_revs,all_rev_score
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,"May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",Action,NaN,19.99,14.99,554,89,42550,92
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,"Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...","Action,Adventure,Massively Multiplayer",Mature Content Description The developers de...,29.99,-1.00,6214,49,836608,49
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,"Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian","Action,Adventure,Strategy",NaN,39.99,-1.00,166,54,7030,71
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,"Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...","Action,Adventure,Massively Multiplayer",NaN,44.99,-1.00,932,57,167115,61
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,"May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French","Action,Free to Play,Massively Multiplayer,RPG,...",NaN,0.00,-1.00,287,54,11481,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,https://store.steampowered.com/app/310490/Croo...,Crookz - The Big Heist,"Aug 25, 2015",Skilltree Studios,"Kalypso Media Digital,Kalypso Media Digital","Strategy,Simulation,Heist,Stealth,Tactical","Single-player,Steam Achievements,Full controll...","English,German,French,Italian,Spanish - Spain,...","Simulation,Strategy",NaN,9.99,-1.00,0,0,134,85
7996,https://store.steampowered.com/app/318330/Mech...,Mecha Ace,"Jun 13, 2014",Choice of Games,"Choice of Games,Choice of Games","Indie,RPG,Choose Your Own Adventure,Mechs,Inte...","Single-player,Steam Achievements,Captions avai...",English,"Indie,RPG",NaN,501.87,424.72,0,0,112,88
7997,https://store.steampowered.com/app/307190/Nicole/,Nicole,"Oct 17, 2013",Winter Wolves,"Winter Wolves,Winter Wolves","Visual Novel,Otome,Simulation,Adventure,Female...","Single-player,Steam Achievements,Steam Trading...",English,"Adventure,Casual,Simulation",NaN,475.60,367.96,0,0,58,81
7998,https://store.steampowered.com/app/304210/Euro...,Euro Truck Simulator 2 - Fantasy Paint Jobs Pack,"Jul 15, 2014",SCS Software,"SCS Software,SCS Software","Simulation,Indie","Single-player,Downloadable Content,Steam Achie...","English,German,French,Italian,Korean,Spanish -...","Indie,Simulation",NaN,1.99,-1.00,0,0,96,89


In [155]:
from collections import Counter 

def remove_duplicates(input): 
    if (type(input) == str):
        # split input string separated by space 
        input = input.split(",") 

        # joins two adjacent elements in iterable way 
        for i in range(0, len(input)): 
            input[i] = "".join(input[i]) 

        # now create dictionary using counter method 
        # which will have strings as key and their 
        # frequencies as value 
        UniqW = Counter(input) 

        # joins two adjacent elements in iterable way 
        s = " ".join(UniqW.keys()) 
        return s
    else:
        s = ""
        return s


purged_steam_data['publisher'] = purged_steam_data.apply(lambda x: remove_duplicates(x['publisher']), axis=1)

purged_steam_data

,url,name,release_date,developer,publisher,popular_tags,game_details,languages,genre,mature_content,original_price,discount_price,amt_rec_revs,rec_rev_score,amt_all_revs,all_rev_score
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,"May 12, 2016",id Software,Bethesda Softworks,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",Action,NaN,19.99,14.99,554,89,42550,92
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,"Dec 21, 2017",PUBG Corporation,PUBG Corporation,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...","Action,Adventure,Massively Multiplayer",Mature Content Description The developers de...,29.99,-1.00,6214,49,836608,49
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,"Apr 24, 2018",Harebrained Schemes,Paradox Interactive,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian","Action,Adventure,Strategy",NaN,39.99,-1.00,166,54,7030,71
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,"Dec 13, 2018",Bohemia Interactive,Bohemia Interactive,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...","Action,Adventure,Massively Multiplayer",NaN,44.99,-1.00,932,57,167115,61
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,"May 6, 2003",CCP,CCP,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French","Action,Free to Play,Massively Multiplayer,RPG,...",NaN,0.00,-1.00,287,54,11481,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,https://store.steampowered.com/app/310490/Croo...,Crookz - The Big Heist,"Aug 25, 2015",Skilltree Studios,Kalypso Media Digital,"Strategy,Simulation,Heist,Stealth,Tactical","Single-player,Steam Achievements,Full controll...","English,German,French,Italian,Spanish - Spain,...","Simulation,Strategy",NaN,9.99,-1.00,0,0,134,85
7996,https://store.steampowered.com/app/318330/Mech...,Mecha Ace,"Jun 13, 2014",Choice of Games,Choice of Games,"Indie,RPG,Choose Your Own Adventure,Mechs,Inte...","Single-player,Steam Achievements,Captions avai...",English,"Indie,RPG",NaN,501.87,424.72,0,0,112,88
7997,https://store.steampowered.com/app/307190/Nicole/,Nicole,"Oct 17, 2013",Winter Wolves,Winter Wolves,"Visual Novel,Otome,Simulation,Adventure,Female...","Single-player,Steam Achievements,Steam Trading...",English,"Adventure,Casual,Simulation",NaN,475.60,367.96,0,0,58,81
7998,https://store.steampowered.com/app/304210/Euro...,Euro Truck Simulator 2 - Fantasy Paint Jobs Pack,"Jul 15, 2014",SCS Software,SCS Software,"Simulation,Indie","Single-player,Downloadable Content,Steam Achie...","English,German,French,Italian,Korean,Spanish -...","Indie,Simulation",NaN,1.99,-1.00,0,0,96,89


In [156]:
#find games that have no popular tags and change to empty to string 

def remove_nans(input):
    if (isinstance(input, str)):
        return input
    else:
        return ""

purged_steam_data['popular_tags'] = purged_steam_data.apply(lambda x: remove_nans(x['popular_tags']), axis=1)

In [186]:
#find number of days since games release relative to today
import datetime
release = datetime.datetime.strptime("May 12, 2016", '%b %d, %Y')
today = datetime.datetime.now()
abs((release - today).days)

1035

In [157]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tif_matrix = vectorizer.fit_transform(purged_steam_data['popular_tags'])
for name in vectorizer.get_feature_names():
    print(name)
print(vectorizer.get_feature_names())
print(tif_matrix.shape)
#tif_matrix

1980s
1990
2d
360
3d
40k
4x
5d
6dof
abstract
access
action
addictive
adventure
agriculture
aliens
alternate
america
and
animation
anime
apocalyptic
arcade
arena
artificial
arts
assassin
asymmetric
asynchronous
atmospheric
attack
atv
audio
automation
awkward
base
baseball
based
basketball
batman
battle
beat
beautiful
benchmark
bikes
blood
bmx
board
book
bowling
builder
building
bullet
campaign
capitalism
card
cartoon
cartoony
casual
cats
character
chess
choices
choose
cinematic
city
class
classic
click
clicker
co
cold
colorful
combat
comedy
comic
competitive
conspiracy
content
control
controller
controls
conversation
crafting
crawler
crime
croft
crowdfunded
crpg
cult
customization
cute
cyberpunk
cycling
dark
dating
death
defense
demons
design
destruction
detective
development
difficult
dinosaurs
diplomacy
documentary
dog
down
dragons
drama
drawn
driving
dungeon
dungeons
dynamic
dystopian
early
economy
editing
editor
education
em
emotional
endings
epic
episodic
events
experience
experime

In [158]:
#find games that have no mature content and change to empty to string 
purged_steam_data['mature_content'] = purged_steam_data.apply(lambda x: remove_nans(x['mature_content']), axis=1)

In [159]:
from sklearn.metrics.pairwise import linear_kernel 
cosine_similarities = linear_kernel(tif_matrix, tif_matrix) 
results = {}
for idx, row in purged_steam_data.iterrows():
   similar_indices = cosine_similarities[idx].argsort()[:-100:-1] #get top 99 most common games
   similar_items = [(cosine_similarities[idx][i], purged_steam_data['url'][i]) for i in similar_indices] 
   results[row['url']] = similar_items[1:]

In [164]:
results[purged_steam_data['url'][1]]
#purged_steam_data['url'][0] ['https://store.steampowered.com/app/782330/DOOM_Eternal/']
#['https://store.steampowered.com/app/718560/Scythe_Digital_Edition/']

[(0.8233990503997944,
  'https://store.steampowered.com/app/813820/Realm_Royale/'),
 (0.7499814574458284,
  'https://store.steampowered.com/app/15120/Tom_Clancys_Rainbow_Six_Vegas_2/'),
 (0.7497570885200636,
  'https://store.steampowered.com/app/460930/Tom_Clancys_Ghost_Recon_Wildlands/'),
 (0.7211355691948168,
  'https://store.steampowered.com/app/555440/Deathgarden_BLOODHARVEST/'),
 (0.7044567029432889,
  'https://store.steampowered.com/app/433850/Z1_Battle_Royale/'),
 (0.6880425437439217,
  'https://store.steampowered.com/app/893520/Creative_Destruction/'),
 (0.6879347540598457,
  'https://store.steampowered.com/app/755790/Ring_of_Elysium/'),
 (0.6827555477703714,
  'https://store.steampowered.com/app/235600/Tom_Clancys_Splinter_Cell_Blacklist/'),
 (0.6668164754299969,
  'https://store.steampowered.com/app/104900/ORION_Prelude/'),
 (0.6596424882738731,
  'https://store.steampowered.com/app/215470/Primal_Carnage/'),
 (0.6578536091396916, 'https://store.steampowered.com/app/222880/Ins

In [167]:
results['https://store.steampowered.com/app/489830/The_Elder_Scrolls_V_Skyrim_Special_Edition/']

[(0.7920010868868157,
  'https://store.steampowered.com/app/611670/The_Elder_Scrolls_V_Skyrim_VR/'),
 (0.7384038796309083,
  'https://store.steampowered.com/app/219780/Divinity_II_Developers_Cut/'),
 (0.6929467233601276,
  'https://store.steampowered.com/app/900883/The_Elder_Scrolls_IV_Oblivion_Game_of_the_Year_Edition_Deluxe/'),
 (0.6929467233601276,
  'https://store.steampowered.com/app/22330/The_Elder_Scrolls_IV_Oblivion_Game_of_the_Year_Edition/'),
 (0.6301359854276765,
  'https://store.steampowered.com/app/288470/Fable_Anniversary/'),
 (0.6202663629698706,
  'https://store.steampowered.com/app/22320/The_Elder_Scrolls_III_Morrowind_Game_of_the_Year_Edition/'),
 (0.6003324891231278,
  'https://store.steampowered.com/app/367500/Dragons_Dogma_Dark_Arisen/'),
 (0.5659228328161767,
  'https://store.steampowered.com/app/204030/Fable__The_Lost_Chapters/'),
 (0.556339297470859, 'https://store.steampowered.com/app/551770/ECHO/'),
 (0.5422135720823295, 'https://store.steampowered.com/app/253